In [1]:
# Import TensorFlow & Mnist dataset
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Implement "load_mnist" function:

import os
import struct
import numpy as np
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte' % kind)
        
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', 
                                 lbpath.read(8))
        labels = np.fromfile(lbpath, 
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", 
                                               imgpath.read(16))
        images = np.fromfile(imgpath, 
                             dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels

In [3]:
#### Loading data from Mnist dataset
X_data, y_data = load_mnist('./mnist', kind='train')
print('Rows: %d, columns: %d' % (X_data.shape[0], X_data.shape[1]))

X_test, y_test = load_mnist('./mnist', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

#### Split training, validation data
X_train, y_train = X_data[:50000, :], y_data[:50000]
X_valid, y_valid = X_data[50000:, :], y_data[50000:]
print('Training: ', X_train.shape, y_train.shape)
print('Training: ', X_valid.shape, y_valid.shape)
print('Training: ', X_test.shape, y_test.shape)

Rows: 60000, columns: 784
Rows: 10000, columns: 784
Training:  (50000, 784) (50000,)
Training:  (10000, 784) (10000,)
Training:  (10000, 784) (10000,)


In [4]:
#### 迭代小批數據的函數
def batch_generator(X, y, batch_size=64, shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
            rng = np.random.RandomState(random_seed)
            rng.shuffle(idx)
            X = X[idx]
            y = y[idx]
            
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [ ]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [ ]:
# 建構"包裝函數"：建構"卷積層"完成必要工作，包括：定義加權、偏誤。初始化後並將使用tf.nn.conv2d函數進行卷積運算
# 有四個必要的參數：
    # input_tensor: "卷積層"的輸入張量
    # name: 用來限定範圍的""

import tensorflow as tf
import numpy as np

def conv_layer(input_tensor, name, kernal_size, n_output_channels, padding_mode='SAME', strides=(1,1,1,1)):
    with tf.variable_scope(name):
        ## get n_input_channels:
        ## input tensor shape:
        ## [batch x width x height x channels-in]
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        weights_shape = list(kernak_size) + \
        [n_input_channels, n_output_channels]
        
        weights = tf.get_variable(name = '_weights ',
                                 shape = weights_shape)
        
        print(weights)
        
        biases = tf.get_variable(name = '_biases',
                                initializer=tf.zeros(
                                shape=[n_output_channels]))
        
        print(biases)
        
        conv = tf.nn.conv2d(input=input_tensor,
                           filter=weights,
                           strides=strides,
                           padding=padding_mode)
        
        print(conv)
        
        conv = tf.nn.bias_add(conv, biases, name='net_pre-activation')
        
        print(conv)
        
        conv = tf.nn.relu(conv, name = 'activation')
        
        return conv
